## handson-ml2-14-cifar10

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds

In [2]:
dataset, info = tfds.load('cifar10', as_supervised=True, shuffle_files=True, with_info=True)

class_names = info.features["label"].names
n_classes = info.features["label"].num_classes
dataset_size = info.splits["train"].num_examples
print(class_names, n_classes, dataset_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling cifar10-train.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling cifar10-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to C:\Users\kun\tensorflow_datasets\cifar10\3.0.2. Subsequent calls will reuse this data.
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 10 50000


In [3]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.inception_v3.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = dataset["train"].map(preprocess).batch(batch_size).prefetch(1)
test_set = dataset["test"].map(preprocess).batch(batch_size).prefetch(1)

In [4]:
a = next(train_set.as_numpy_iterator())
print(a[0].shape, a[1].shape, a[1])

(32, 224, 224, 3) (32,) [7 8 4 4 6 5 2 9 6 6 9 9 3 0 8 7 9 0 4 9 0 8 6 4 2 8 8 7 0 8 4 2]


In [5]:
model = keras.Sequential([
    keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(n_classes, activation='softmax'),
])
# for i, layer in enumerate(model.layers[0].layers):
#   print(i, layer.name)
for layer in model.layers[0].layers[:249]:
   layer.trainable = False
for layer in model.layers[0].layers[249:]:
   layer.trainable = True

model.compile(optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9),
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])
model.summary()

87924736/87910968 [==============================] - 51s 1us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 23,911,210
Trainable params: 13,223,306
Non-trainable params: 10,687,904
_________________________________________________________________


C:\Users\kun\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [6]:
model.fit(train_set, validation_data=test_set, batch_size=batch_size, epochs=7)

Epoch 1/7
1563/1563 [==============================] - 81s 49ms/step - loss: 0.9527 - accuracy: 0.7426 - val_loss: 0.4942 - val_accuracy: 0.8497
Epoch 2/7
1563/1563 [==============================] - 75s 48ms/step - loss: 0.4540 - accuracy: 0.8556 - val_loss: 0.3926 - val_accuracy: 0.8725
Epoch 3/7
1563/1563 [==============================] - 76s 49ms/step - loss: 0.3696 - accuracy: 0.8796 - val_loss: 0.3532 - val_accuracy: 0.8846
Epoch 4/7
1563/1563 [==============================] - 74s 48ms/step - loss: 0.3154 - accuracy: 0.8976 - val_loss: 0.3309 - val_accuracy: 0.8910
Epoch 5/7
1563/1563 [==============================] - 75s 48ms/step - loss: 0.2727 - accuracy: 0.9126 - val_loss: 0.3167 - val_accuracy: 0.8941
Epoch 6/7
1563/1563 [==============================] - 113s 73ms/step - loss: 0.2366 - accuracy: 0.9250 - val_loss: 0.3073 - val_accuracy: 0.8970
Epoch 7/7
1563/1563 [==============================] - 219s 140ms/step - loss: 0.2047 - accuracy: 0.9366 - val_loss: 0.3013 - val